# Импорт библиотек

In [ ]:
!pip install bokeh
!pip install selenium
!conda install -c conda-forge firefox geckodriver
import pandas as pd
import bokeh
from bokeh.models import Title
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from typing import Union, List, Optional
import numpy as np

output_notebook()



/bin/bash: line 1: conda: command not found
Cloning into 'my_works'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.
/content/my_works/my_works/my_works


# Рабочая среда



In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Vlados_work/draft/base_data.parquet', )
df.rename(columns={'open_futures': 'open',
                   'high_futures': 'high',
                   'low_futures': 'low',
                   'close_futures': 'close',
                   'first_volume_futures': 'volume',
                   'timestamp': 'datetime'},
                   inplace=True)

# Преобразование столбца 'datetime' в тип данных datetime
df['date'] = df.datetime.dt.date

df = df[:5000]

In [ ]:
def customize_plot_styles(p):
    p.background_fill_color = "#181c27"
    p.background_fill_alpha = 1

    p.xaxis.major_label_orientation = "horizontal"

    p.xaxis.axis_label_text_color = "#b2b5be"
    p.xaxis.major_label_text_color = "#b2b5be"
    p.xaxis.axis_line_width = 1.0
    p.xaxis.axis_line_color = "#2a2e39"

    p.yaxis.minor_tick_line_color = None
    p.xaxis.minor_tick_line_color = None

    p.yaxis.axis_label_text_color = "#b2b5be"
    p.yaxis.major_label_text_color = "#b2b5be"
    p.yaxis.axis_line_width = 1.0
    p.yaxis.axis_line_color = "#2a2e39"

    p.xaxis.major_tick_line_color = "#2a2e39"
    p.yaxis.major_tick_line_color = "#2a2e39"

    p.border_fill_color = "#181c27"
    p.grid.grid_line_color = "#2a2e39"
    p.outline_line_color = "#2a2e39"

    p.xgrid.grid_line_dash = [2, 2]
    p.ygrid.grid_line_dash = [2, 2]
    p.xgrid.grid_line_alpha = 0
    p.ygrid.grid_line_alpha = 0

    p.xaxis.major_tick_line_color = None  # Убрать линии тиков по оси X
    p.yaxis.major_tick_line_color = None  # Убрать линии тиков по оси Y

    p.toolbar.autohide = True





In [ ]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import Span, LabelSet, HoverTool, Slider, DatePicker, Toggle, CrosshairTool, FreehandDrawTool
from bokeh.layouts import column, row
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.callbacks import CustomJS
import pandas as pd
import numpy as np
from typing import Optional, List, Union


def customize_plot_styles(p):
    p.background_fill_color = "#181c27"
    p.background_fill_alpha = 1
    p.xaxis.major_label_orientation = "horizontal"
    p.xaxis.axis_label_text_color = "#b2b5be"
    p.xaxis.major_label_text_color = "#b2b5be"
    p.xaxis.axis_line_width = 1.0
    p.xaxis.axis_line_color = "#2a2e39"
    p.yaxis.minor_tick_line_color = None
    p.xaxis.minor_tick_line_color = None
    p.yaxis.axis_label_text_color = "#b2b5be"
    p.yaxis.major_label_text_color = "#b2b5be"
    p.yaxis.axis_line_width = 1.0
    p.yaxis.axis_line_color = "#2a2e39"
    p.border_fill_color = "#181c27"
    p.grid.grid_line_color = "#2a2e39"
    p.outline_line_color = "#2a2e39"
    p.xgrid.grid_line_dash = [2, 2]
    p.ygrid.grid_line_dash = [2, 2]
    p.xgrid.grid_line_alpha = 0
    p.ygrid.grid_line_alpha = 0
    p.xaxis.major_tick_line_color = None  # Убрать линии тиков по оси X
    p.yaxis.major_tick_line_color = None  # Убрать линии тиков по оси Y
    p.toolbar.autohide = True

def create_data_sources(df: pd.DataFrame):
    """Функция для создания источников данных для графиков"""

    # Копируем датафрейм
    df = df.copy()

    # Вычисляем центральную точку каждой свечи
    df['middle'] = (df['open'] + df['close']) / 2

    # Вычисляем высоту и ширину каждой свечи
    df['height_inc'] = df['close'] - df['open']
    df['height_dec'] = df['open'] - df['close']

    # Вычисляем цвет каждой свечи
    inc = df['close'] > df['open']
    dec = df['open'] > df['close']

    # Создаем источники данных
    df_inc = ColumnDataSource(df[inc])
    df_dec = ColumnDataSource(df[dec])
    df_source = ColumnDataSource(df)

    return df, df_inc, df_dec, df_source


def process_positions(df: pd.DataFrame,
                      positions_df: Optional[pd.DataFrame] = None,
                      open_position_timestamps: Optional[List[int]] = None,
                      close_position_timestamps: Optional[List[int]] = None,
                      stop_losses: Optional[List[float]] = None,
                      marker_sell_stop_color: str='red',
                      marker_sell_color: str='green',):
    """Функция для обработки позиций и создания источников данных для графиков"""

    # Если positions_df не передан, то используем списки open_position_timestamps, close_position_timestamps и stop_losses
    if positions_df is not None:
        required_position_columns = ['open_position_timestamp', 'close_position_timestamp', 'stop_loss']
        for col in required_position_columns:
            if col not in positions_df.columns:
                raise ValueError(f"DataFrame с метками позиций должен содержать столбец '{col}'")
    else:
        if open_position_timestamps is None or close_position_timestamps is None or stop_losses is None:
            raise ValueError("Необходимо передать либо positions_df, либо списки open_position_timestamps, close_position_timestamps и stop_losses")
        positions_df = pd.DataFrame({
            'open_position_timestamp': open_position_timestamps,
            'close_position_timestamp': close_position_timestamps,
            'stop_loss': stop_losses
        })

    # Преобразование дат в строковый формат
    positions_df['open_position_timestamp'] = pd.to_datetime(positions_df['open_position_timestamp'], unit='s')
    positions_df['close_position_timestamp'] = pd.to_datetime(positions_df['close_position_timestamp'], unit='s')


    # Вычисляем индексы позиций
    indices_buy = df.index[df['datetime'].isin(positions_df['open_position_timestamp'])].tolist()
    indices_sell = df.index[df['datetime'].isin(positions_df['close_position_timestamp'])].tolist()


    # Создаем источники данных
    num_candles_in_trade = []
    adjusted_sell_indices = []
    percent_moves = []
    open_prices = []
    close_prices = []


    # Обрабатываем позиции
    for i, buy_idx in enumerate(indices_buy):
        # Итерируемся по позициям
        if i < len(indices_sell):
            sell_idx = indices_sell[i]
            stop_loss = positions_df['stop_loss'].iloc[i]
            # Определяем есть ли стоп лосс
            if stop_loss is not None:
                stop_loss_hit = False
                for j in range(buy_idx, sell_idx):
                    if df['close'].iloc[j] < stop_loss:
                        sell_idx = j
                        stop_loss_hit = True
                        break
                if stop_loss_hit:
                    color = marker_sell_stop_color
                else:
                    color = marker_sell_color
            else:
                color = marker_sell_color

            # Записываем данные
            num_candles_in_trade.append(sell_idx - buy_idx)
            percent_move = ((df['close'].iloc[sell_idx] - df['open'].iloc[buy_idx]) / df['open'].iloc[buy_idx]) * 100
            percent_moves.append(percent_move)
            adjusted_sell_indices.append((sell_idx, color))
            open_prices.append(df['open'].iloc[buy_idx])
            close_prices.append(df['close'].iloc[sell_idx])
        else:
            num_candles_in_trade.append(len(df) - buy_idx)
            adjusted_sell_indices.append((None, marker_sell_color))
            percent_moves.append(None)
            open_prices.append(df['open'].iloc[buy_idx])
            close_prices.append(None)

    return indices_buy, adjusted_sell_indices, num_candles_in_trade, percent_moves, open_prices, close_prices


def create_markers_data(df, indices_buy, adjusted_sell_indices, num_candles_in_trade, percent_moves, open_prices, close_prices,
                        distance_buy, distance_sell, marker_buy_color, marker_sell_color, marker_sell_stop_color):
    marker_data_buy = {
        'datetime': df['datetime'][indices_buy],
        'y': df['low'][indices_buy] - distance_buy,
        'labels': np.arange(1, len(indices_buy) + 1).astype(str),
        'color': [marker_buy_color] * len(indices_buy),
        'num_candles': num_candles_in_trade
    }

    marker_data_sell = {
        'datetime': [],
        'y': [],
        'labels': [],
        'color': [],
        'num_candles': [],
        'percent_move': [],
        'open_price': [],
        'close_price': []
    }

    for i, buy_idx in enumerate(indices_buy):
        sell_idx, color = adjusted_sell_indices[i]
        if sell_idx is not None:
            marker_data_sell['datetime'].append(df['datetime'].iloc[sell_idx])
            marker_data_sell['y'].append(df['high'].iloc[sell_idx] + distance_sell)
            marker_data_sell['labels'].append(str(i + 1))
            marker_data_sell['num_candles'].append(sell_idx - buy_idx)
            marker_data_sell['color'].append(color)
            marker_data_sell['percent_move'].append(percent_moves[i])
            marker_data_sell['open_price'].append(df['open'].iloc[buy_idx])
            marker_data_sell['close_price'].append(df['close'].iloc[sell_idx])

    return marker_data_buy, marker_data_sell


def plot_trading_signals(
        df: pd.DataFrame,
        positions_df: Optional[pd.DataFrame] = None,
        open_position_timestamps: Optional[List[int]] = None,
        close_position_timestamps: Optional[List[int]] = None,
        stop_losses: Optional[List[float]] = None,
        distance_buy: int = 5,
        distance_sell: int = 5,
        marker_buy_color: str = 'blue',
        marker_sell_color: str = 'green',
        marker_sell_stop_color: str = 'red',
        buy_marker: str = 'triangle',
        sell_marker: str = 'inverted_triangle',
        chart_width: int = 1000,
        chart_height: int = 400,
        profit_candle_color: str = '#0072B2',
        loss_candle_color: str = 'white',
        marker_size: int = 10,
        labels_color: str = 'white',
        candle_spacing: str = '35sec',
        width_slider_start: int = 200,
        width_slider_end: int = 1400,
        width_slider_step: int = 10,
        width_slider_value: int = 400,
        height_slider_value: int = 300,
        height_slider_start: int = 200,
        height_slider_end: int = 600,
        height_slider_step: int = 10,
        tools: str = 'wheel_zoom,pan,box_zoom,reset,save,undo',
        color_segment: str = '#787b86',
        y_axis_location: str = 'right',
        toolbar_location: str = 'above',
        line_width_curr: float = 0.5,
        line_color_curr: str = '#787b86',
        line_dash_curr: Union[str, List[int]] = 'dashed',
        horizontal_line_dash: Union[str, List[int]] = [4, 4],
        horizontal_line_color: str = '#2a2e39',
        horizontal_line_width: float = 1,
        show_horizontal_line: bool = True,
        title: Optional[str] = None,
        start_date_picker_title: str = 'Начальная дата',
        end_date_picker_title: str = 'Конечная дата',
        legend_location: str = 'top_right',
        legend_title: Optional[str] = None,
        legend_title_text_font_style: str = 'italic',
        legend_title_text_font_size: str = '10px',
        legend_label_text_color: str = "#b2b5be",
        legend_label_text_font: str = 'Helvetica',
        legend_label_text_font_style: str = 'normal',
        legend_label_text_font_size: str = '12px',
        legend_background_fill_color: str = '#181c27',
        legend_border_line_color: str = '#2a2e39',
        legend_border_line_width: float = 1,
        legend_border_line_alpha: float = 0.5,
        legend_background_fill_alpha: float = 0.5,
        title_text_color: str = '#b2b5be',
        marker_legend_label_buy: str = 'buy_signal',
        marker_legend_label_sell: str = 'sell_signal',
) -> figure:
    """
    Создает график с торговыми сигналами на основе данных свечей с временными метками открытия/закрытия позиций и стоп-лоссами.

    df: DataFrame с данными свечей (должны содержать столбцы 'datetime', 'open', 'high', 'low', 'close').
    positions_df: DataFrame с временными метками открытия и закрытия позиций, а также стоп-лоссами (должен содержать столбцы 'open_position_timestamp', 'close_position_timestamp', 'stop_loss').
    open_position_timestamps: Список временных меток открытия позиций.
    close_position_timestamps: Список временных меток закрытия позиций.
    stop_losses: Список стоп-лоссов для каждой сделки.
    distance_buy: Дистанция от свечи для маркеров покупок.
    distance_sell: Дистанция от свечи для маркеров продаж.
    marker_buy_color: Цвет маркеров покупок.
    marker_sell_color: Цвет маркеров продаж.
    marker_sell_stop_color: Цвет маркеров продаж при срабатывании стоп-лосса.
    buy_marker: Форма маркеров покупок.
    sell_marker: Форма маркеров продаж.
    chart_width: Ширина графика. По умолчанию 1000.
    chart_height: Высота графика. По умолчанию 400.
    loss_candle_color: Цвет свечей при убытке цены.
    profit_candle_color: Цвет свечей при возрастании цены.
    marker_size: Размер маркеров. По умолчанию 10.
    labels_color: Цвет меток. По умолчанию 'white'.
    candle_spacing: Расстояние между свечами. По умолчанию '35sec'.
    width_slider_start: Начальное значение ширины графика в виджете Slider. По умолчанию 200.
    width_slider_end: Конечное значение ширины графика в виджете Slider. По умолчанию 1400.
    width_slider_step: Шаг изменения ширины графика в виджете Slider. По умолчанию 10.
    width_slider_value: Значение по умолчанию для ширины графика в виджете Slider. По умолчанию 400.
    height_slider_value: Значение по умолчанию для высоты графика в виджете Slider. По умолчанию 300.
    height_slider_start: Начальное значение высоты графика в виджете Slider. По умолчанию 200.
    height_slider_end: Конечное значение высоты графика в виджете Slider. По умолчанию 600.
    height_slider_step: Шаг изменения высоты графика в виджете Slider. По умолчанию 10.
    tools: Набор инструментов для графика. По умолчанию "wheel_zoom,pan,box_zoom,reset,save,undo".
    color_segment: настройка цвета усиков свечей.
    y_axis_location: настройка расположения оси y. По умолчанию 'right'.
    toolbar_location: Настройка расположения набора инструментов. По умолчанию сверху.
    line_width_curr: ширина указательной линии. По умолчанию 0.5.
    line_color_curr: цвет указательной линии. По умолчанию '#787b86'.
    line_dash_curr: стиль линии. По умолчанию 'dashed'.
    horizontal_line_dash: стиль горизонтальной линии. По умолчанию [4, 4].
    horizontal_line_color: цвет горизонтальной линии. По умолчанию '#2a2e39'.
    horizontal_line_width: ширина горизонтальной линии. По умолчанию 1.
    show_horizontal_line: показывать ли горизонтальную линию. По умолчанию True.
    title: Заголовок графика. По умолчанию None.
    start_date_picker_title: Название кнопки для выбора начальной даты. По умолчанию 'Начальная дата'.
    end_date_picker_title: Название кнопки для выбора конечной даты. По умолчанию 'Конечная дата'.
    legend_location: расположение легенды. По умолчанию 'top_right'.
    legend_title: Заголовок легенды. По умолчанию None.
    legend_title_text_font_style: Стиль шрифта заголовка легенды. По умолчанию "italic".
    legend_title_text_font_size: Размер шрифта заголовка легенды. По умолчанию "10px".
    legend_label_text_color: Цвет меток легенды. По умолчанию "#b2b5be".
    legend_label_text_font: Шрифт меток легенды. По умолчанию "Helvetica".
    legend_label_text_font_style: Стиль шрифта меток легенды. По умолчанию "normal".
    legend_label_text_font_size: Размер шрифта меток легенды. По умолчанию "12px".
    legend_background_fill_color: Цвет фона легенды. По умолчанию '#181c27'.
    legend_border_line_color: Цвет границы легенды. По умолчанию '#2a2e39'.
    legend_border_line_width: Ширина границы легенды. По умолчанию 1.
    legend_border_line_alpha: Прозрачность границы легенды. По умолчанию 1.
    legend_background_fill_alpha: Прозрачность фона легенды. По умолчанию 0.5.
    title_text_color: Цвет заголовка. По умолчанию '#b2b5be'.
    marker_legend_label_buy: Название маркера покупки. По умолчанию 'buy_signal'.
    marker_legend_label_sell: Название маркера продажи. По умолчанию 'sell_signal'.

    Returns:
        figure с графиком.
    """
    # Проверка типов данных
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Параметр 'df' должен быть pandas DataFrame.")
    if positions_df is not None and not isinstance(positions_df, pd.DataFrame):
        raise TypeError("Параметр 'positions_df' должен быть pandas DataFrame.")
    if positions_df is None and (open_position_timestamps is None or close_position_timestamps is None or stop_losses is None):
        raise ValueError("Необходимо передать либо positions_df, либо списки open_position_timestamps, close_position_timestamps и stop_losses")

    # Создаем источники данных
    df, df_inc, df_dec, df_source = create_data_sources(df)

    # Обрабатываем позиции
    indices_buy, adjusted_sell_indices, num_candles_in_trade, percent_moves, open_prices, close_prices = process_positions(
        df, positions_df, open_position_timestamps, close_position_timestamps, stop_losses)

    # Создаем данные для маркеров
    marker_data_buy, marker_data_sell = create_markers_data(
        df, indices_buy, adjusted_sell_indices, num_candles_in_trade, percent_moves, open_prices, close_prices,
        distance_buy, distance_sell, marker_buy_color, marker_sell_color, marker_sell_stop_color)

    # Создаем график
    width1 = Span(dimension="width", line_dash=line_dash_curr, line_width=line_width_curr, line_color=line_color_curr)
    height1 = Span(dimension="height", line_dash=line_dash_curr, line_width=line_width_curr, line_color=line_color_curr)

    p = figure(width=chart_width,
               height=chart_height,
               tools=[tools, CrosshairTool(overlay=[width1, height1])],
               y_axis_location=y_axis_location,
               toolbar_location=toolbar_location,
               active_scroll='wheel_zoom',
               active_drag='pan',
               x_axis_type="datetime",
               toolbar_sticky=False,
               title=title,
               )

    customize_plot_styles(p)

    # Создаем мультилинию, используя источник данных
    source = ColumnDataSource(data=dict(xs=[], ys=[]))
    multi_line = p.multi_line('xs', 'ys', source=source)

    tool = FreehandDrawTool(renderers=[multi_line])
    p.add_tools(tool)

    # Добавление горизонтальной линии
    average_price = df['close'].mean()
    horizontal_line = Span(location=average_price, dimension='width', line_dash=horizontal_line_dash,
                           line_color=horizontal_line_color, line_width=horizontal_line_width)
    p.add_layout(horizontal_line)

    # Добавление свечей и сегментов
    p.segment(x0='datetime', y0='high', x1='datetime', y1='low', source=df_source, color=color_segment)
    p.rect(x='datetime', y='middle', width=pd.Timedelta(candle_spacing), height='height_inc', source=df_inc,
           fill_color=profit_candle_color, line_color=profit_candle_color)
    p.rect(x='datetime', y='middle', width=pd.Timedelta(candle_spacing), height='height_dec', source=df_dec,
           fill_color=loss_candle_color, line_color=loss_candle_color)

    # Добавление маркеров на график
    marker_source_buy = ColumnDataSource(data=marker_data_buy)
    marker_source_sell = ColumnDataSource(data=marker_data_sell)
    buy_glyph = p.scatter(x='datetime', y='y', source=marker_source_buy, size=marker_size, color='color', marker=buy_marker,
                          legend_label=marker_legend_label_buy)
    sell_glyph = p.scatter(x='datetime', y='y', source=marker_source_sell, size=marker_size, color='color', marker=sell_marker,
                           legend_label=marker_legend_label_sell)

    # Добавление меток для маркеров
    labels_buy = LabelSet(x='datetime', y='y', text='labels', level='glyph', x_offset=0, y_offset=0,
                          source=marker_source_buy, text_color=labels_color, text_font_size='5pt', text_align='center',
                          text_baseline='middle')
    labels_sell = LabelSet(x='datetime', y='y', text='labels', level='glyph', x_offset=0, y_offset=0,
                           source=marker_source_sell, text_color=labels_color, text_font_size='5pt', text_align='center',
                           text_baseline='middle')
    p.add_layout(labels_buy)
    p.add_layout(labels_sell)

    # Добавление HoverTool для маркеров
    hover_buy = HoverTool(renderers=[buy_glyph], tooltips=[("Кол-во свечей в сделке", "@num_candles")])
    hover_sell = HoverTool(renderers=[sell_glyph],
                           tooltips=[
                               ("Дата и время", "@datetime{%F %T}"),
                               ("Кол-во свечей в сделке", "@num_candles"),
                               ("Процент движения", "@percent_move{0.00} %"),
                               ("Цена открытия", "@open_price{0,0}"),
                               ("Цена закрытия", "@close_price{0,0}")
                           ],
                           formatters={'@datetime': 'datetime', '@{adj close}': 'printf'})
    p.add_tools(hover_buy)
    p.add_tools(hover_sell)

    # Настройка формата даты на оси x
    p.xaxis.formatter = DatetimeTickFormatter(
        hours="%H:%M",
        minutes="%H:%M",
        days="%d %b",
        months="%b %Y"
    )

    hover = HoverTool(mode='vline', renderers=[p.renderers[1]])
    hover.tooltips = [
        ("datetime", "@datetime{%F %T}"),
        ("open", "@open{0,0}"),
        ("high", "@high{0,0}"),
        ("low", "@low{0,0}"),
        ("close", "@close{0,0}"),
        ("volume", "@volume{0.00 a} m"),
        ("index", "@index")
    ]
    hover.formatters = {'@datetime': 'datetime', '@{adj close}': 'printf'}
    hover.mode = "mouse"
    p.add_tools(hover)

    # Создание виджетов и добавление их к графику
    width_slider = Slider(start=width_slider_start, end=width_slider_end, step=width_slider_step, value=width_slider_value)
    width_slider.js_link('value', p, 'width')

    height_slider = Slider(start=height_slider_start, end=height_slider_end, step=height_slider_step, value=height_slider_value)
    height_slider.js_link('value', p, 'height')

    start_date_str = df['datetime'].iloc[0].strftime('%Y-%m-%d')
    end_date_str = df['datetime'].iloc[-1].strftime('%Y-%m-%d')

    start_date_picker = DatePicker(title=start_date_picker_title, value=start_date_str)
    end_date_picker = DatePicker(title=end_date_picker_title, value=end_date_str)

    callback = CustomJS(args=dict(p=p, start_date_picker=start_date_picker, end_date_picker=end_date_picker), code="""
        var start_date = new Date(start_date_picker.value).getTime();
        var end_date = new Date(end_date_picker.value).getTime();
        p.x_range.start = start_date;
        p.x_range.end = end_date;
    """)

    start_date_picker.js_on_change('value', callback)
    end_date_picker.js_on_change('value', callback)

    toggle1 = Toggle(label="horizontal_line", button_type="success", active=True)
    toggle1.js_link('active', horizontal_line, 'visible')

    toggle_buy = Toggle(label="markers_buy", button_type="success", active=True)
    toggle_buy.js_link('active', buy_glyph, 'visible')
    toggle_buy.js_link('active', labels_buy, 'visible')

    toggle_sell = Toggle(label="markers_sell", button_type="success", active=True)
    toggle_sell.js_link('active', sell_glyph, 'visible')
    toggle_sell.js_link('active', labels_sell, 'visible')

    p.legend.location = legend_location
    p.legend.title = legend_title
    p.legend.title_text_font_style = legend_title_text_font_style
    p.legend.title_text_font_size = legend_title_text_font_size
    p.legend.label_text_color = legend_label_text_color
    p.legend.label_text_font = legend_label_text_font
    p.legend.label_text_font_style = legend_label_text_font_style
    p.legend.label_text_font_size = legend_label_text_font_size
    p.legend.background_fill_color = legend_background_fill_color
    p.legend.background_fill_alpha = legend_background_fill_alpha
    p.legend.border_line_width = legend_border_line_width
    p.legend.border_line_color = legend_border_line_color
    p.legend.border_line_alpha = legend_border_line_alpha

    if title is not None:
        p.title.text_color = title_text_color

    layout = column(row(start_date_picker, end_date_picker), width_slider, height_slider, row(toggle1, toggle_buy, toggle_sell), p)
    show(layout)


# Пример использования функции
open_timestamps = [1577103060, 1577104980, 1577107680]
close_timestamps = [1577104020, 1577105820, None]
stop_losses = [7543, 7541, None]

# Предполагается, что `df` уже создан и содержит необходимые данные
plot_trading_signals(df, open_position_timestamps=open_timestamps, close_position_timestamps=close_timestamps, stop_losses=stop_losses)
